<a href="https://colab.research.google.com/github/Npkh/BlueStars-Tools/blob/main/Auto_L%C3%A0m_D%E1%BB%AF_Li%E1%BB%87u_B%C3%A1o_C%C3%A1o_Dashboard_Canamax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách Thực Hiện Auto Làm Dữ Liệu Dashboard**
*   B1: Tải File checklist sản phẩm về (file này là checklist sản phẩm, có thể cần cập nhật trong tương lai hoặc trong quá trình lọc thiếu sản phẩm. File này chỉ tải về 1 lần duy nhất là lần đầu sử dụng code. Sau đó lưu và cập nhật tên sản phẩm trực tiếp vào file này trong tương lai)

[***File checklist sản phẩm***](https://docs.google.com/spreadsheets/d/1LdJtTZxRaRQurNjtGYs6KfFgG031N_Xz/edit?usp=sharing&ouid=105336776683915595895&rtpof=true&sd=true)

*   B2: Copy từng dữ liệu của data (SB, SD, SP) sau khi tải về vào sheet "data", sheet này cần được clear data mỗi lần thực hiện xong báo cáo

*   B3: Lưu và upload file này lên trên đoạn code này, bên dưới có option "Upload file"

*   B4: Kết quả tải về cần format lại định dạng một số cột như:
  - Date, Lauching time theo format cần thiết
  - Budget, Spend, CPC, Sale sang dạng currency

*   B5: Kiểm tra cuối cùng đoạn code để biết có cần cập nhật checklist hay không

***Lưu Ý***:
*   Đoạn code sẽ tự động chạy và tải về kết quả, nếu không có kết quả tải về thì đoạn code đã bị lỗi, cần check lại các bước đã làm đúng hay chưa. Thời gian chạy code khoảng 30s-40s

*   Nếu chẳng may xóa một đoạn code nào đó, reload lại trang này để khôi phục code

*   Phải đăng nhập google drive mới có thể chạy Run all

*   Các cột có giá trị là **currency** đã được tự động *0.76 khi nhập market là Canada

*   Làm từng data một, không copy hết vào chung sheet data. Làm xong data nào thì clear data ở sheet "data" và paste data mới vào


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import files
import warnings

In [ ]:
market_input = input("Nhập thị trường tương ứng (United States or Canada): ")
ad_type = input("Nhập ad_type tương ứng (SP, SD, SB): ")
file_name_output = input("Nhập tên file Excel muốn lưu: ")
uploaded = files.upload()

Nhập thị trường tương ứng (United States or Canada): Canada
Nhập ad_type tương ứng (SP, SD, SB): SP
Nhập tên file Excel muốn lưu: a


Saving Checklist Canamax.xlsx to Checklist Canamax (4).xlsx


In [ ]:
dfs = {}

for file_name in uploaded.keys():
    xls = pd.ExcelFile(file_name)
    sheet_names = xls.sheet_names

    for sheet_name in sheet_names:
        dfs[sheet_name] = pd.read_excel(file_name, sheet_name=sheet_name)

product_id = dfs['product_info']
report = dfs['data']

In [ ]:
product_id['product_id'] = product_id['product_id'].astype(str)
report['Campaign Name'] = report['Campaign Name'].astype(str)
def find_product_number(campaign_name):
    matching_product_numbers = []

    for product_number in product_id['product_id']:
        product_words = product_number.split()
        words_to_match = campaign_name.split()
        matching_sequence = ""
        product_idx = 0

        if words_to_match[0].lower() == "xmars" and words_to_match[1].lower() == "ai":
            words_to_match = words_to_match[2:]

        for word in words_to_match:
            if product_idx < len(product_words) and word.lower() == product_words[product_idx].lower():
                matching_sequence += word + " "
                product_idx += 1
            else:
                break

        if matching_sequence.strip().lower() == product_number.lower():
            matching_product_numbers.append(product_number)

    if matching_product_numbers:
        matching_product_numbers.sort(key=lambda x: len(x), reverse=True)
        return matching_product_numbers[0]

    return None
report['Product Number'] = report['Campaign Name'].apply(find_product_number)

In [ ]:
def get_matching_sku(campaign_name):
    for sku in product_id['SKU']:
        if sku in campaign_name:
            return sku
    return None

report['SKU'] = report['Campaign Name'].apply(get_matching_sku)

In [ ]:
trim_column_names = lambda x: x.strip()
report.rename(columns=trim_column_names, inplace=True)

if ad_type == 'SP':
    report.rename(columns={'Click-Thru Rate (CTR)':'CTR',
                     'Cost Per Click (CPC)':'CPC',
                     '7 Day Total Orders (#)':'Orders',
                     'Total Advertising Cost of Sales (ACOS)':'ACOS',
                     'Total Return on Advertising Spend (ROAS)':'ROAS',
                     '7 Day Total Sales':'Sales'}, inplace=True)
else:
    report.rename(columns={'Click-Thru Rate (CTR)':'CTR',
                         'Cost Per Click (CPC)':'CPC',
                         '14 Day Total Orders (#)':'Orders',
                         'Total Advertising Cost of Sales (ACOS)':'ACOS',
                         'Total Return on Advertising Spend (ROAS)':'ROAS',
                         '14 Day Total Sales':'Sales'}, inplace=True)

In [ ]:
if ad_type != "SB" and report['Budget'].dtype == 'object':

    report['Budget'] = report['Budget'].astype(str)

    report['Budget'] = report['Budget'].str.replace("CA", "").str.replace("US", "").str.replace("$", "")

    report['Budget'] = report['Budget'].astype(float)

if report['Spend'].dtype == 'object':

    report['Spend'] = report['Spend'].astype(str)

    report['Spend'] = report['Spend'].str.replace("CA", "").str.replace("US", "").str.replace("$", "")

    report['Spend'] = report['Spend'].astype(float)

if report['CPC'].dtype == 'object':

    report['CPC'] = report['CPC'].astype(str)

    report['CPC'] = report['CPC'].str.replace("CA", "").str.replace("US", "").str.replace("$", "")

    report['CPC'] = report['CPC'].astype(float)

if report['Sales'].dtype == 'object':

    report['Sales'] = report['Sales'].astype(str)

    report['Sales'] = report['Sales'].str.replace("CA", "").str.replace("US", "").str.replace("$", "")

    report['Sales'] = report['Sales'].astype(float)

<ipython-input-34-7b148b4c3319>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  report['Budget'] = report['Budget'].str.replace("CA", "").str.replace("US", "").str.replace("$", "")
<ipython-input-34-7b148b4c3319>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  report['Spend'] = report['Spend'].str.replace("CA", "").str.replace("US", "").str.replace("$", "")
<ipython-input-34-7b148b4c3319>:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  report['CPC'] = report['CPC'].str.replace("CA", "").str.replace("US", "").str.rep

In [ ]:
report['Product Number'] = report['Product Number'].astype(str)
report['SKU'] = report['SKU'].astype(str)

In [ ]:
report['Cost Type'] = report['Campaign Name'].apply(lambda x: 'CPM' if 'CPM' in x else 'CPC')

if ad_type == "SP":
  def classify_campaign_name(campaign_name):
      if "Auto" in campaign_name:
          return "Auto"
      elif "Query" in campaign_name:
          return "SP Query"
      elif "jido" in campaign_name:
          return "Auto"
      elif "Research" in campaign_name:
          return "Research"
      elif "Performance" in campaign_name:
          return "Performance"
      elif "term" in campaign_name:
          return "search terms"
      elif " TD" in campaign_name:
          return "TD"
      elif "Broad" in campaign_name:
          return "SP Broad"
      elif "Exact" in campaign_name:
          return "SP Exact"
      elif "TOS" in campaign_name:
          return "TOS"
      elif " PP" in campaign_name:
          return "PP"
      elif " PT" in campaign_name:
          return "SP PT"
      elif "EX8" in campaign_name:
          return "SP Exact"
      else:
          return "SP Phrase"

elif ad_type == "SB":
  def classify_campaign_name(campaign_name):
      if "Video Ads Phrase" in campaign_name:
          return "SB Video Phrase"
      elif "Video Ads Broad" in campaign_name:
          return "SB Video Broad"
      elif "SB" in campaign_name:
          return "SB"
      elif "Video Ads PT" in campaign_name:
          return "SB Video PT"
      elif "Video Ads Query" in campaign_name:
          return "SB Video Query"
      else:
          return "SB Video Exact"

if ad_type == "SD":
  report['Campaign Form'] = 'SD PT'
else:
  report['Campaign Form'] = report['Campaign Name'].apply(classify_campaign_name)

if market_input == "Canada":
  report['Spend'] = report['Spend'] * 0.76
  report['Sales'] = report['Sales'] * 0.76

report['Market'] = market_input
report['Brand'] = "Canamax"

if ad_type == "SB":
  report["Campaign Type"] = 'Sponsor Brands'
  report["Status"] = np.nan
  report["Budget"] = np.nan
  report["Targeting Type"] = np.nan
  report["Bidding strategy"] = np.nan

elif ad_type == "SD":
  report["Campaign Type"] = 'Sponsor Display'
  report["Targeting Type"] = np.nan
  report["Bidding strategy"] = np.nan

def classify_value(value):
    if '&' in value:
        return "Combo"
    elif 'PO' in value:
        return "Pack"
    else:
        return "Single"

report['Product Type'] = report['Product Number'].apply(classify_value)
report['Start Date'] = np.nan
report['End Date'] = np.nan

order1 = ['Date', 'Campaign Type', 'Campaign Name','Bidding strategy',
         'Impressions','Clicks', 'Spend',
         'Orders', 'Sales', 'Product Number',
         'Campaign Form', 'Market', 'Brand', 'Cost Type', 'SKU']

data_merge1 = report[order1]

In [ ]:
if 'Product Number' in report.columns and 'SKU' in report.columns:
    product_number_none = (report['Product Number'] == 'None').any()
    sku_none = (report['SKU'] == 'None').any()

    if product_number_none and sku_none:
        print("WARNING: Cả hai cột Product Number và SKU đều có giá trị None, cần kiểm tra lại!!!")
    elif product_number_none:
        print("WARNING: Cột 'Product Number' có giá trị None, cần kiểm tra lại!!!")
    elif sku_none:
        print("WARNING: Cột 'SKU' có giá trị None, cần kiểm tra lại!!!")
    else:
        print("Dữ liệu hoàn chỉnh. Không cần kiểm tra")
else:
    print("Không tìm thấy cột 'Product Number' hoặc 'SKU' trong báo cáo.")

Dữ liệu hoàn chỉnh. Không cần kiểm tra


In [ ]:
with pd.ExcelWriter(file_name_output + ".xlsx") as writer:
    data_merge1.to_excel(writer, sheet_name='Final data', index=False)

files.download(file_name_output + ".xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>